<a href="https://colab.research.google.com/github/text123580/ml20200716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/AI_04/test0724/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/AI_04/test0724/chinese_news_test.zip")
f.extractall("test")

In [9]:
import glob
import os
import pandas as pd

def getdf(targetdn):
    datas = {
        "path":[],
        "label":[]
    }
    base = ""
    p = os.path.join(base, targetdn, "*")
    for dn in glob.glob(p):
        name = os.path.split(dn)[-1]
        fn = os.path.join(dn, "*.[tT][xX][tT]")
        fns = glob.glob(fn)
        datas["label"] = datas["label"] + [name] * len(fns)
        datas["path"] = datas["path"] + fns
    return pd.DataFrame(datas)

In [10]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

In [11]:
def read_content(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)

In [12]:
train_df

,path,label,content
0,train/chinese_news_trans/軍事/813.TXT,軍事,古巴宣佈舉行軍事演習\n新華社哈瓦那５月１日電古巴革命武裝部今晚發表公...
1,train/chinese_news_trans/軍事/8223.txt,軍事,\n美國空軍改進B-1B轟炸機的電子戰系統(附圖) \n\n----------------...
2,train/chinese_news_trans/軍事/8123.TXT,軍事,１枚導彈致１２０名阿富汗人傷亡\n新華社伊斯蘭堡５月２２日電阿富汗游擊...
3,train/chinese_news_trans/軍事/871.TXT,軍事,馬達加斯加發生軍事政變\n新華社塔那那利佛５月１３日電據馬達加斯加電臺...
4,train/chinese_news_trans/軍事/889.TXT,軍事,緬甸取消部份地區軍事管制\n新華社仰光５月１５日電據緬甸之聲電臺報道，...
...,...,...,...
2632,train/chinese_news_trans/政治/954.TXT,政治,李鵬同奧其爾巴特舉行會談\n新華社北京５月４日電（記者楊子迪）國務院總...
2633,train/chinese_news_trans/政治/9463.TXT,政治,多米尼加共和國中止大選計票工作\n新華社北京５月２１日電聖多明各消息：...
2634,train/chinese_news_trans/政治/9195.TXT,政治,泰國公主瓦塔娜離泰訪華\n新華社曼谷５月１０日電泰國公主甘拉亞妮·瓦塔...
2635,train/chinese_news_trans/政治/9444.TXT,政治,伊駐英使館稱收到裝有電子元件的匿名信\n新華社倫敦５月２１日電（記者王...


In [37]:
u = train_df["label"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trans)
y_test = test_df["label"].replace(trans)

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: label, Length: 101, dtype: int64

In [38]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.721 seconds.
Prefix dict has been built successfully.


0      美海軍 成功 演示 飛機 在 空中 控制 無人機 ( 附圖 )   \n \n ------...
1      \n 軍事 評論 ： “ 巴格達 之戰 ” 考驗 美軍 巷戰 武器裝備 ( 2 )   \n...
2      軍事 評論 ： 美英 聯軍 地面 作戰 裝備 特點 分析 ( 2 )   \n \n ---...
3      在 伊拉克 戰爭 中 使用 的 空襲 精確 制導 彈藥 ( 1 )   \n \n ----...
4      軍事 評論 ： 從 巷戰 或 地道戰 看 單兵 夜視 裝備 的 發展   \n \n ---...
                             ...                        
96                           楊 主席 提出 促進 中 拉關係 的 ５ 項 建...
97                           阿拉伯 外長 會議 閉幕 \n 新華社 巴格達 ...
98                           外事 簡訊 \n 新華社 北京 ５ 月 ２ ４ ...
99                           第 ３ 次 安第斯集團 首腦會議 閉幕 \n 新...
100                  Ａ Ａ ０ ５ ５ \n 金日成 指出 朝鮮 北方 堅持 以 聯...
Name: content, Length: 101, dtype: object

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [41]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [42]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [43]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      columns=[n+"(預測)" for n in u],
      index=[n+"(真實)" for n in u])

,軍事(預測),交通(預測),計算機(預測),環境(預測),藝術(預測),醫藥(預測),教育(預測),經濟(預測),體育(預測),政治(預測)
軍事(真實),10,0,0,0,0,0,0,0,0,0
交通(真實),0,10,0,0,0,0,0,0,0,0
計算機(真實),0,0,10,0,0,0,0,0,0,0
環境(真實),0,0,0,10,0,0,0,0,0,0
藝術(真實),0,0,0,0,10,0,0,0,0,0
醫藥(真實),0,0,0,0,0,10,0,0,0,0
教育(真實),0,0,0,0,0,0,10,0,0,0
經濟(真實),0,0,0,0,0,0,0,10,0,0
體育(真實),0,0,0,0,0,0,0,0,10,0
政治(真實),0,0,0,0,0,0,0,0,0,11


In [44]:
test = input("輸入一篇新聞:")
count = vec.transform([newscut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
    print(name, "的機率:", p)
print("答案:", trans_r[answer])

輸入一篇新聞:美國媒體報導，美國國務卿蓬佩奧訪問倫敦期間告訴英國國會議員，有可靠情資顯示中國與譚德塞達成一項協議，以助他當上世衛秘書長；這項協議也影響世衛決策，導致2019冠狀病毒疾病（COVID-19）疫情期間有許多英國人喪命。（看完整報導）
軍事 的機率: 0.8696426870922035
交通 的機率: 5.921834555115776e-09
計算機 的機率: 0.007874033477415792
環境 的機率: 1.5758417133688261e-06
藝術 的機率: 1.869200062734955e-10
醫藥 的機率: 5.081090366278561e-11
教育 的機率: 1.4836619012147777e-10
經濟 的機率: 0.08069028706545726
體育 的機率: 1.2051484537374535e-09
政治 的機率: 0.0417914090101138
答案: 軍事
